In [12]:
## Import necessary libraries

from bs4 import BeautifulSoup

import requests

import pandas as pd
import numpy as np

import csv

## Request the html source from web-address

source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

#source.status_code # to check returned text is html or not

## Parse the source using BeautifulSoup

soup=BeautifulSoup(source, 'lxml')

## find the table tag in html

table=soup.find('table')
table

## Create csv file, write headers in it, find all 'tr' tags and then find all 'td' tags and store them in list and write list into file. Last close the file

with open('Toronto.csv','w') as csv_file:
    csv_writer=csv.writer(csv_file)
    csv_writer.writerow(['PostalCode','Borough','Neighborhood'])
    for tb in table.find_all('tr'):
        list1=[]
        for head in tb.find_all('td'):
            list1.append(head.text)
        csv_writer.writerow(list1)

csv_file.close()

## Read csv file, drop rows with Borough=Not assigned

df=pd.read_csv('Toronto.csv')

df=df[df.Borough!='Not assigned']

df.head(10)

df1=df.reset_index(drop=True) # reset index to 0

## Replace the unnecessary characters from Neighborhood column

df1.Neighborhood=df1.Neighborhood.str.replace("\\r\n","") 
df1.head(10)

## Set Neighborhood=Borough where Neighborhood is 'Not assigned'

df1['Neighborhood'].loc[df1['Neighborhood']=='Not assigned']=df1['Borough']
df1.head(10)

df1.shape # find shape

## Combine the rows with same PostalCode

res = df1.groupby(['PostalCode','Borough'])['Neighborhood'].agg( ','.join).reset_index()

res

res.shape #check the shape

(103, 3)

In [13]:

df2=pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

df2.head()

res1=pd.concat([res,df2],axis=1)

res1.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n,Malvern\n",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,M1H,43.773136,-79.239476


In [3]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(res1['Borough'].unique()),
        res1['Neighborhood'].shape
    )
)

The dataframe has 11 boroughs and (103,) neighborhoods.


In [4]:
from geopy.geocoders import Nominatim 
address = 'Toronto, On'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


## Creating a map of Toronto with neighborhoods superimposed on top

In [38]:
# create map of New York using latitude and longitude values
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(res1['Latitude'], res1['Longitude'], res1['Borough'], res1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

ModuleNotFoundError: No module named 'folium'

In [6]:
#toronto_data = res1[res1.Borough.replace({'Toronto':np.nan},regex=True).isnull()]
downtown_data = res1[res1['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.Neighborhood=downtown_data.Neighborhood.str.replace("\\n","") 
downtown_data

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,M4W,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",M4X,43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",M5B,43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",M5H,43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",M5J,43.640816,-79.381752


In [14]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [15]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

NameError: name 'folium' is not defined

In [16]:
pc_data

NameError: name 'pc_data' is not defined

In [17]:
CLIENT_ID = 'OC0GPD4H3HWLICOX3Y243SO4XMFAUFSV2E5GR4USPUPUSBZE' # your Foursquare ID
CLIENT_SECRET = 'OHV5JOGVGHWQAQQVRCA303CL2YQYVV2QLYLIFEN0JNXP3RTG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OC0GPD4H3HWLICOX3Y243SO4XMFAUFSV2E5GR4USPUPUSBZE
CLIENT_SECRET:OHV5JOGVGHWQAQQVRCA303CL2YQYVV2QLYLIFEN0JNXP3RTG


In [18]:
downtown_data.loc[0, 'Neighborhood']

'Rosedale'

In [19]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [20]:
LIMIT = 100  # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=OC0GPD4H3HWLICOX3Y243SO4XMFAUFSV2E5GR4USPUPUSBZE&client_secret=OHV5JOGVGHWQAQQVRCA303CL2YQYVV2QLYLIFEN0JNXP3RTG&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2c4e1cbf7dde002c95f27b'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [27]:
print(downtown_venues.shape)
downtown_venues.head()

(1297, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [28]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,89,89,89,89,89,89
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [29]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 206 uniques categories.


# 3.Analyse each neighborhood

In [30]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
downtown_onehot.shape

(1297, 206)

In [32]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.00,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.011236,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.00,0.011236,0.000000,0.000000,0.011236,0.000000
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.01,0.00,0.0,0.00,0.060000,0.000000,0.050000,0.010000,0.000000
6,Christie,0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011364,0.00,0.011364,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.011364,0.00,0.00,0.0,0.00,0.000000,0.011364,0.011364,0.000000,0.011364
8,"Commerce Court,Victoria Hotel",0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.00,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,...,0.000000,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [33]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3            Beer Bar  0.04
4  Seafood Restaurant  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.19
1  Airport Terminal  0.12
2    Airport Lounge  0.12
3   Harbor / Marina  0.06
4  Sculpture Garden  0.06


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.04
2       Bakery  0.04
3          Pub  0.04
4       Market  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2      Ice Cream Shop  0.04
3  Italian Restaurant  0.04
4     

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,American Restaurant,Gym,Asian Restaurant,Sushi Restaurant,Bakery
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Steakhouse,Seafood Restaurant,Farmers Market,Cheese Shop,Café,Liquor Store
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Boat or Ferry,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Bar
3,"Cabbagetown,St. James Town",Coffee Shop,Italian Restaurant,Bakery,Market,Pet Store,Pizza Place,Café,Restaurant,Pub,Diner
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Chinese Restaurant,Bubble Tea Shop,Salad Place,Spa


## 4.Cluster Neighborhood

In [36]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 1, 3, 4, 1, 1, 1], dtype=int32)

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,M4W,43.679563,-79.377529,2,Park,Building,Playground,Trail,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",M4X,43.667967,-79.367675,1,Coffee Shop,Italian Restaurant,Bakery,Market,Pet Store,Pizza Place,Café,Restaurant,Pub,Diner
2,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160,1,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Hotel,Gastropub,Pub,Fast Food Restaurant
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Gym / Fitness Center,Café,Theater,Restaurant,Breakfast Spot
4,M5B,Downtown Toronto,"Ryerson,Garden District",M5B,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Middle Eastern Restaurant,Restaurant,Diner,Italian Restaurant,Japanese Restaurant
